# 1. 파일 불러오기

In [2]:
import pandas as pd

df_from_excel = pd.read_excel('./고등교육기관 하반기 주소록(2021).xlsx', engine='openpyxl') #

df_from_excel.columns = df_from_excel.loc[4].tolist()

df_from_excel = df_from_excel.drop(index=list(range(0,5)))  # 데이터의 0~5줄 삭제

print(df_from_excel.head())

print(df_from_excel['학교명'].values)  # 학교명 데이터 모두 출력

print(df_from_excel['주소'].values)  # 주소 데이터 모두 출력

     연도       학교종류  시도      행정구        학교명                           학교명(영문)  \
5  2021  전문대학(4년제)  서울  서울 동대문구    삼육보건대학교        Sahmyook Health University   
6  2021  전문대학(4년제)  서울  서울 서대문구  서울여자간호대학교  Seoul Women's College of nursing   
7  2021  전문대학(4년제)  부산   부산 사하구      동주대학교                    Dongju College   
8  2021  전문대학(4년제)  광주    광주 남구    기독간호대학교      Christian College of Nursing   
9  2021  전문대학(4년제)  광주    광주 동구    조선간호대학교            CHOSUN NURSING COLLEGE   

  본분교 학교상태  설립   우편번호                                  주소  \
5  본교   기존  사립  02500             서울특별시 동대문구 망우로 82 (휘경동)   
6  본교   기존  사립  03617  서울특별시 서대문구 간호대로 38 (홍제동, 서울여자간호대학)   
7  본교   기존  사립  49318    부산광역시 사하구 사리로55번길 16 (괴정동, 동주대학)   
8  본교   기존  사립  61662    광주광역시 남구 백서로70번길 6 (양림동, 기독간호대학)   
9  본교   기존  사립  61453  광주광역시 동구 필문대로 309-2 (서석동, 조선간호대학교)   

                   전화번호                  팩스번호                  홈페이지  
5  02-2212-0082          02-2215-2380          http://www.shu.ac.kr  
6  02-2287-1

# 2. 지오코더 API 사용 (브이월드 -> 오픈 API)

- 지오코딩 : 기존 주소를 이용하여 위도, 경도 값을 구하는 법
- https://mskim8717.tistory.com/73
- https://bskyvision.com/entry/python-%EC%A3%BC%EC%86%8C%EB%A5%BC-%EC%9C%84%EB%8F%84-%EA%B2%BD%EB%8F%84-%EC%A2%8C%ED%91%9C%EB%A1%9C-%EB%B0%94%EA%BF%94%EC%A3%BC%EB%8A%94-Geocoder-API-%EC%82%AC%EC%9A%A9%EB%B2%95

In [3]:
# API 접속내용

import requests

url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'   #도로명주소
road_type2 = 'PARCEL' #지번주소
address = '&address='
keys = '&key='
primary_key = 'ADE5750E-D419-3486-95FC-803D986D236A'  # 발급받은 본인의 인증키

# ------------------------------------------------------------------------------------------------------------------------------

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y

x,y = request_geo("경기도 안산시 상록구 한양대학로 55 (한양대학교 ERICA캠퍼스)")

print(f'x값: {x}')  # 경도값
print(f'y값: {y}')  # 위도값

x값: 126.839531145
y값: 37.298063894


### 가입이 필요없는 지오코딩 : https://parkgihyeon.github.io/project/geocoding-api/

# 3. API를 이용하여 엑셀 데이터의 값을 위도, 경도 값 변환

In [4]:
import pandas as pd
import requests
from openpyxl import load_workbook
from openpyxl import Workbook
import re


df_from_excel = pd.read_excel('./고등교육기관 하반기 주소록(2021).xlsx', engine='openpyxl')
df_from_excel.columns = df_from_excel.loc[4].tolist()
df_from_excel = df_from_excel.drop(index=list(range(0,5)))


url = 'http://api.vworld.kr/req/address?'
params = 'service=address&request=getcoord&version=2.0&crs=epsg:4326&refine=true&simple=false&format=json&type='
road_type = 'ROAD'   #도로명주소
road_type2 = 'PARCEL' #지번주소
address = '&address='
keys = '&key='
primary_key = 'ADE5750E-D419-3486-95FC-803D986D236A'

def request_geo(road):
    page = requests.get(url+params+road_type+address+road+keys+primary_key)
    json_data = page.json()
    if json_data['response']['status'] == 'OK':
        x = json_data['response']['result']['point']['x']
        y = json_data['response']['result']['point']['y']
        return x,y
    else:
        x = 0
        y = 0
        return x,y


try:
    wb = load_workbook(r"./학교주소좌표.xlsx", data_only=True)
    sheet  = wb.active
except:
    wb = Workbook()
    sheet = wb.active

university_list = df_from_excel['학교명'].to_list()
address_list = df_from_excel['주소'].to_list()

for num,value in enumerate(address_list):
    addr = re.sub(r'\([^)]*\)', '', value)  #  주소부분에서 괄호 부분 삭제
    print(addr)
    x,y = request_geo(addr)
    sheet.append([university_list[num],addr,x,y])  # 데이터를 학교명, 주소, x, y 의 순서대로 엑셀에 저장

wb.save(r"./학교주소좌표.xlsx")

서울특별시 동대문구 망우로 82 
서울특별시 서대문구 간호대로 38 
부산광역시 사하구 사리로55번길 16 
광주광역시 남구 백서로70번길 6 
광주광역시 동구 필문대로 309-2 
대전광역시 동구 백룡로 59 
강원도 영월군 영월읍 하송로 197 
충청북도 음성군 감곡면 대학길 278 
충청남도 천안시 동남구 문암로 58 
전라북도 군산시 동개정길 7 
전라남도 영암군 학산면 영산로 76-57 
전라남도 순천시 제일대학길 17 
서울특별시 구로구 경인로 445 
서울특별시 서대문구 가좌로 134 
서울특별시 종로구 필운대로1길 34 
서울특별시 노원구 초안산로 12 
부산광역시 사상구 주례로 45 
대구광역시 북구 영송로 15 
대구광역시 수성구 달구벌대로528길 15 
인천광역시 계양구 계양산로 63 
인천광역시 미추홀구 인하로 100 
광주광역시 광산구 북문대로419번길 73 
광주광역시 북구 동문대로 50 
광주광역시 북구 서강로 1 
대전광역시 서구 혜천로 100 
울산광역시 동구 봉수로 101 
울산광역시 울주군 웅촌면 대학길 9 
세종특별자치시 장군면 대학길 300 
경기도 의정부시 서부로 545 
경기도 남양주시 진접읍 경복대로 425 
경기도 김포시 월곶면 김포대학로 97 
경기도 고양시 덕양구 서삼릉길 281 
경기도 안양시 동안구 임곡로 29 
경기도 수원시 장안구 천천로74번길 50 
경기도 안성시 삼죽면 동아예대길 47 
경기도 광주시 곤지암읍 경충대로 26 
경기도 안산시 단원구 예술대학로 171 
경기도 양주시 은현면 서정로 27 
경기도 화성시 정남면 세자로 288 
경기도 성남시 중원구 광명로 377 
경기도 안산시 상록구 안산대학로 155 
경기도 안양시 만안구 양화로37번길 34 
경기도 오산시 청학로 45 
경기도 용인시 처인구 동부로 61 
경기도 파주시 탄현면 웅지로144번길 73 
경기도 이천시 마장면 청강가창로 389-94 
경기도 평택시 삼남로 283 
강원도 강릉시 공제로 357 
강원도 강릉시 주문진읍 연주로 2

KeyboardInterrupt: 

# 4. 지도에 마커 표시

In [6]:
import folium

map = folium.Map(location=[37,127],zoom_start=7)  # 처음보여주는 위도와 경도 설정

# 위도와 경도에 팝업 이름을 파란색의 마커 아이콘으로 표시
marker = folium.Marker([37.298063894, 126.839531145],
                    popup='한양대학교 ERICA캠퍼스', 
                    icon = folium.Icon(color='blue'))

marker.add_to(map) 

map.save(r'./uni1_map.html')  # 다음 파일명으로 저장

In [7]:
import pandas as pd
import folium

df_from_excel = pd.read_excel('./학교주소좌표.xlsx', engine='openpyxl',header=None)

df_from_excel.columns = ['학교이름','주소','x','y']

name_list = df_from_excel['학교이름'].to_list()
addr_list = df_from_excel['주소'].to_list()
position_x_list = df_from_excel['x'].to_list()
position_y_list = df_from_excel['y'].to_list()

map = folium.Map(location=[37,127],zoom_start=7)

for i in range(len(name_list)):
    if position_x_list[i] != 0:  # x값(경도값)이 0이 아닐때만 참
        marker = folium.Marker([position_y_list[i],position_x_list[i]],
                            popup=name_list[i], 
                            icon = folium.Icon(color='blue'))
        marker.add_to(map) 

map.save(r'./uni2_map.html')